In [1]:
import pandas as pd
import xarray as xr 
from dask_jobqueue import SLURMCluster

cluster = SLURMCluster(
    job_name="Climt1",          # --job-name
    cores=48,                   # 24 cores per node
    processes=4,                # One process per task
    memory="400GB",             # --mem
    walltime="04:00:00",        # --time
    queue="med",                # --partition
    log_directory=".",          # Logs will be saved to the current directory
)

/home1/nalex2023/.local/lib/python3.10/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 39129 instead
  warnings.warn(


In [2]:
from dask.distributed import Client


cluster.scale(jobs=1)
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: http://10.42.239.61:39129/status,
Dashboard: http://10.42.239.61:39129/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.42.239.61:41861,Workers: 0
Dashboard: http://10.42.239.61:39129/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [32]:
twp_manus = '/home2/nalex2023/Datasets/LSB_OUTS/met_manus_30min/'

sun_dft = pd.read_csv('/home1/nalex2023/main/LSB_main/LSB_detectionv2/Datasets/sunrise_sunset_twp.csv')

sun_dft = sun_dft.map(pd.to_datetime).set_index('date')

wind_speed_ds = xr.open_mfdataset(twp_manus+"*.nc").sel(time=slice('1999-12-30','2014'))

In [40]:
sun_dft['sunrise_mask'] = sun_dft['sunrise(UTC)'].round('1h') 

sun_dft

,sunrise(UTC),sunset(UTC),sunrise_mask
date,,,
2000-01-01,1999-12-31 20:06:38.444661376,2000-01-01 08:20:39.908245248,1999-12-31 20:06:38.444661376
2000-01-02,2000-01-01 20:07:07.708542720,2000-01-02 08:21:07.342338560,2000-01-01 20:07:07.708542720
2000-01-03,2000-01-02 20:07:36.726734976,2000-01-03 08:21:34.371180672,2000-01-02 20:07:36.726734976
2000-01-04,2000-01-03 20:08:05.464764288,2000-01-04 08:22:00.962267264,2000-01-03 20:08:05.464764288
2000-01-05,2000-01-04 20:08:33.888653824,2000-01-05 08:22:27.083652992,2000-01-04 20:08:33.888653824
...,...,...,...
2014-12-28,2014-12-27 20:04:48.841008640,2014-12-28 08:18:55.510619648,2014-12-27 20:04:48.841008640
2014-12-29,2014-12-28 20:05:18.684226048,2014-12-29 08:19:24.142050560,2014-12-28 20:05:18.684226048
2014-12-30,2014-12-29 20:05:48.366424320,2014-12-30 08:19:52.445321472,2014-12-29 20:05:48.366424320


In [36]:
filter_apply_resutls = pd.DataFrame()

filter_apply_resutls['days'] = pd.date_range(start='2000-01-01', end='2014-12-31', freq='D')

sunrise_da = sun_dft['sunrise(UTC)'].reindex(wind_speed_ds.time)


sunrise_da

1999-12-30 00:00:00   NaT
1999-12-30 00:30:00   NaT
1999-12-30 01:00:00   NaT
1999-12-30 01:30:00   NaT
1999-12-30 02:00:00   NaT
                       ..
2013-12-31 21:30:00   NaT
2013-12-31 22:00:00   NaT
2013-12-31 22:30:00   NaT
2013-12-31 23:00:00   NaT
2013-12-31 23:30:00   NaT
Name: sunrise(UTC), Length: 245510, dtype: datetime64[ns]

In [ ]:
def filter1_wind_speed(wind_speed,sun_dft):
    date = wind_speed.time.dt.date.values.astype(str)
    sunrise = sun_dft['sunrise(UTC)'].loc[date]
    sunset = sun_dft['sunset(UTC)'].loc[date]
    be